# 📦 Data Collection - Madagascar Vanilla Price Prediction

Ce notebook collecte les données nécessaires pour la prédiction du prix de la vanille.

## Sources de données
1. **World Bank Pink Sheet** - Prix mensuels des commodités
2. **FAOSTAT** - Production et exportations
3. **Données supplémentaires** - Taux de change, climat

In [1]:
# Imports
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW_DATA_PATH = Path('../data/raw')
PROCESSED_DATA_PATH = Path('../data/processed')

print("✅ Imports successful")

✅ Imports successful


## 1. World Bank Commodity Prices (Pink Sheet)

Téléchargement des prix mensuels historiques des commodités incluant la vanille.

In [2]:
# URL du fichier World Bank (mise à jour novembre 2024)
WORLD_BANK_MONTHLY_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
WORLD_BANK_ANNUAL_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Annual.xlsx"

def download_world_bank_data():
    """Télécharge les données World Bank Pink Sheet"""
    
    # Télécharger données mensuelles
    print("📥 Téléchargement des données mensuelles World Bank...")
    monthly_path = RAW_DATA_PATH / 'world_bank_monthly.xlsx'
    
    response = requests.get(WORLD_BANK_MONTHLY_URL)
    if response.status_code == 200:
        with open(monthly_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Données mensuelles sauvegardées: {monthly_path}")
    else:
        print(f"❌ Erreur téléchargement: {response.status_code}")
        return None
    
    # Télécharger données annuelles
    print("📥 Téléchargement des données annuelles World Bank...")
    annual_path = RAW_DATA_PATH / 'world_bank_annual.xlsx'
    
    response = requests.get(WORLD_BANK_ANNUAL_URL)
    if response.status_code == 200:
        with open(annual_path, 'wb') as f:
            f.write(response.content)
        print(f"✅ Données annuelles sauvegardées: {annual_path}")
    else:
        print(f"❌ Erreur téléchargement: {response.status_code}")
    
    return monthly_path, annual_path

# Télécharger
paths = download_world_bank_data()

📥 Téléchargement des données mensuelles World Bank...
✅ Données mensuelles sauvegardées: ../data/raw/world_bank_monthly.xlsx
📥 Téléchargement des données annuelles World Bank...
✅ Données annuelles sauvegardées: ../data/raw/world_bank_annual.xlsx


In [3]:
# Charger et explorer les données World Bank
monthly_path = RAW_DATA_PATH / 'world_bank_monthly.xlsx'

# Lire le fichier Excel - voir les sheets disponibles
xl = pd.ExcelFile(monthly_path)
print("📋 Sheets disponibles:")
for sheet in xl.sheet_names:
    print(f"  - {sheet}")

📋 Sheets disponibles:
  - AFOSHEET
  - Monthly Prices
  - Monthly Indices
  - Description
  - Index Weights


In [4]:
# Charger la sheet avec les prix mensuels
# Note: Le nom exact de la sheet peut varier, ajuster si nécessaire
try:
    # Essayer différents noms possibles
    for sheet_name in ['Monthly Prices', 'Monthly', 'Prices']:
        if sheet_name in xl.sheet_names:
            df_prices = pd.read_excel(monthly_path, sheet_name=sheet_name)
            print(f"✅ Chargé depuis sheet: {sheet_name}")
            break
    else:
        # Si aucun match, prendre la première sheet
        df_prices = pd.read_excel(monthly_path, sheet_name=0)
        print(f"✅ Chargé depuis première sheet")
except Exception as e:
    print(f"❌ Erreur: {e}")

print(f"\n📊 Shape: {df_prices.shape}")
df_prices.head(10)

✅ Chargé depuis sheet: Monthly Prices

📊 Shape: (796, 72)


,World Bank Commodity Price Data (The Pink Sheet),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,"monthly prices in nominal US dollars, 1960 to ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(monthly series are available only in nominal ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Updated on November 04, 2025",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African **","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
4,NaN,($/bbl),($/bbl),($/bbl),($/bbl),($/mt),($/mt),($/mmbtu),($/mmbtu),($/mmbtu),...,($/mt),($/dmtu),($/mt),($/mt),($/mt),($/mt),($/mt),($/troy oz),($/troy oz),($/troy oz)
5,NaN,CRUDE_PETRO,CRUDE_BRENT,CRUDE_DUBAI,CRUDE_WTI,COAL_AUS,COAL_SAFRICA,NGAS_US,NGAS_EUR,NGAS_JP,...,ALUMINUM,IRON_ORE,COPPER,LEAD,Tin,NICKEL,Zinc,GOLD,PLATINUM,SILVER
6,1960M01,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,...,511.471832,11.42,715.4,206.1,2180.4,1631,260.8,35.27,83.5,0.9137
7,1960M02,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,...,511.471832,11.42,728.19,203.7,2180.4,1631,244.9,35.27,83.5,0.9137
8,1960M03,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,...,511.471832,11.42,684.94,210.3,2173.8,1631,248.7,35.27,83.5,0.9137
9,1960M04,1.63,1.63,1.63,…,…,…,0.14,0.404774,…,...,511.471832,11.42,723.11,213.6,2178.2,1631,254.6,35.27,83.5,0.9137


In [5]:
# Rechercher la colonne vanille
print("🔍 Recherche de colonnes contenant 'vanilla':")
vanilla_cols = [col for col in df_prices.columns if 'vanilla' in str(col).lower()]
print(vanilla_cols)

print("\n📋 Toutes les colonnes:")
for i, col in enumerate(df_prices.columns):
    print(f"{i}: {col}")

🔍 Recherche de colonnes contenant 'vanilla':
[]

📋 Toutes les colonnes:
0: World Bank Commodity Price Data (The Pink Sheet)
1: Unnamed: 1
2: Unnamed: 2
3: Unnamed: 3
4: Unnamed: 4
5: Unnamed: 5
6: Unnamed: 6
7: Unnamed: 7
8: Unnamed: 8
9: Unnamed: 9
10: Unnamed: 10
11: Unnamed: 11
12: Unnamed: 12
13: Unnamed: 13
14: Unnamed: 14
15: Unnamed: 15
16: Unnamed: 16
17: Unnamed: 17
18: Unnamed: 18
19: Unnamed: 19
20: Unnamed: 20
21: Unnamed: 21
22: Unnamed: 22
23: Unnamed: 23
24: Unnamed: 24
25: Unnamed: 25
26: Unnamed: 26
27: Unnamed: 27
28: Unnamed: 28
29: Unnamed: 29
30: Unnamed: 30
31: Unnamed: 31
32: Unnamed: 32
33: Unnamed: 33
34: Unnamed: 34
35: Unnamed: 35
36: Unnamed: 36
37: Unnamed: 37
38: Unnamed: 38
39: Unnamed: 39
40: Unnamed: 40
41: Unnamed: 41
42: Unnamed: 42
43: Unnamed: 43
44: Unnamed: 44
45: Unnamed: 45
46: Unnamed: 46
47: Unnamed: 47
48: Unnamed: 48
49: Unnamed: 49
50: Unnamed: 50
51: Unnamed: 51
52: Unnamed: 52
53: Unnamed: 53
54: Unnamed: 54
55: Unnamed: 55
56: Unnamed: 5

In [6]:
# Extraire les données vanille
# Adapter selon la structure réelle du fichier

def extract_vanilla_prices(df):
    """
    Extrait les prix de la vanille du DataFrame World Bank.
    La structure peut nécessiter des ajustements.
    """
    
    # Chercher l'index de début des données (souvent après quelques lignes d'en-tête)
    # et la colonne vanille
    
    # Option 1: Si les données sont bien structurées avec dates en index
    # Option 2: Si la première colonne contient les dates
    
    # Afficher les premières lignes pour comprendre la structure
    print("Structure des données:")
    print(df.iloc[:5, :5])
    
    return df

df_vanilla_raw = extract_vanilla_prices(df_prices)

Structure des données:
    World Bank Commodity Price Data (The Pink Sheet)          Unnamed: 1  \
0  monthly prices in nominal US dollars, 1960 to ...                 NaN   
1  (monthly series are available only in nominal ...                 NaN   
2                       Updated on November 04, 2025                 NaN   
3                                                NaN  Crude oil, average   
4                                                NaN             ($/bbl)   

         Unnamed: 2        Unnamed: 3      Unnamed: 4  
0               NaN               NaN             NaN  
1               NaN               NaN             NaN  
2               NaN               NaN             NaN  
3  Crude oil, Brent  Crude oil, Dubai  Crude oil, WTI  
4           ($/bbl)           ($/bbl)         ($/bbl)  


## 2. Données alternatives - Création de dataset synthétique

Si les données World Bank ne contiennent pas directement la vanille, nous créons un dataset basé sur les prix historiques connus.

In [7]:
def create_vanilla_dataset():
    """
    Crée un dataset de prix de vanille basé sur les données historiques connues.
    Sources: FAO, rapports industrie, articles de presse
    
    Prix en USD/kg pour la vanille de Madagascar (gousses)
    """
    
    # Données historiques approximatives des prix de la vanille (USD/kg)
    # Basées sur rapports FAO et analyses de marché
    
    historical_data = {
        '2010': 25,
        '2011': 30,
        '2012': 25,
        '2013': 20,
        '2014': 80,    # Début de la hausse
        '2015': 120,
        '2016': 250,   # Cyclone + spéculation
        '2017': 500,   # Pic historique
        '2018': 600,   # Maximum
        '2019': 450,   # Début baisse
        '2020': 350,   # COVID impact
        '2021': 250,
        '2022': 200,
        '2023': 180,
        '2024': 150,
    }
    
    # Créer série mensuelle avec variation saisonnière
    dates = pd.date_range(start='2010-01-01', end='2024-12-01', freq='MS')
    
    prices = []
    for date in dates:
        year = str(date.year)
        base_price = historical_data.get(year, 150)
        
        # Ajouter saisonnalité (prix plus hauts en juin-août après récolte)
        month = date.month
        if month in [6, 7, 8]:
            seasonal_factor = 1.1  # +10% post-récolte
        elif month in [1, 2, 3]:
            seasonal_factor = 0.95  # -5% début d'année
        else:
            seasonal_factor = 1.0
        
        # Ajouter bruit aléatoire
        noise = np.random.normal(0, base_price * 0.05)
        
        price = base_price * seasonal_factor + noise
        prices.append(max(10, price))  # Prix minimum 10 USD
    
    df = pd.DataFrame({
        'date': dates,
        'price_usd_kg': prices
    })
    
    return df

# Créer le dataset
np.random.seed(42)  # Pour reproductibilité
df_vanilla = create_vanilla_dataset()

print(f"📊 Dataset créé: {len(df_vanilla)} observations")
print(f"📅 Période: {df_vanilla['date'].min()} à {df_vanilla['date'].max()}")
df_vanilla.head(10)

📊 Dataset créé: 180 observations
📅 Période: 2010-01-01 00:00:00 à 2024-12-01 00:00:00


,date,price_usd_kg
0,2010-01-01,24.370893
1,2010-02-01,23.577170
2,2010-03-01,24.559611
3,2010-04-01,26.903787
4,2010-05-01,24.707308
5,2010-06-01,27.207329
6,2010-07-01,29.474016
7,2010-08-01,28.459293
8,2010-09-01,24.413157
9,2010-10-01,25.678200


In [8]:
# Statistiques descriptives
print("📈 Statistiques des prix de la vanille (USD/kg):")
df_vanilla['price_usd_kg'].describe()

📈 Statistiques des prix de la vanille (USD/kg):


count    180.000000
mean     217.992423
std      182.872761
min       17.040330
25%       31.448859
50%      186.352208
75%      329.397999
max      656.585577
Name: price_usd_kg, dtype: float64

## 3. Ajout de features supplémentaires

In [9]:
def add_features(df):
    """
    Ajoute des features temporelles et économiques
    """
    df = df.copy()
    
    # Features temporelles
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    
    # Indicateur de saison de récolte (mai-juillet)
    df['harvest_season'] = df['month'].isin([5, 6, 7]).astype(int)
    
    # Indicateur de saison cyclonique (janvier-mars)
    df['cyclone_season'] = df['month'].isin([1, 2, 3]).astype(int)
    
    # Lag features
    df['price_lag1'] = df['price_usd_kg'].shift(1)
    df['price_lag3'] = df['price_usd_kg'].shift(3)
    df['price_lag6'] = df['price_usd_kg'].shift(6)
    df['price_lag12'] = df['price_usd_kg'].shift(12)
    
    # Moyennes mobiles
    df['price_ma3'] = df['price_usd_kg'].rolling(window=3).mean()
    df['price_ma6'] = df['price_usd_kg'].rolling(window=6).mean()
    df['price_ma12'] = df['price_usd_kg'].rolling(window=12).mean()
    
    # Variation mensuelle
    df['price_pct_change'] = df['price_usd_kg'].pct_change()
    
    # Volatilité (écart-type sur 6 mois)
    df['price_volatility'] = df['price_usd_kg'].rolling(window=6).std()
    
    return df

df_vanilla_features = add_features(df_vanilla)
print(f"📊 Nombre de features: {len(df_vanilla_features.columns)}")
df_vanilla_features.head(15)

📊 Nombre de features: 16


,date,price_usd_kg,year,month,quarter,harvest_season,cyclone_season,price_lag1,price_lag3,price_lag6,price_lag12,price_ma3,price_ma6,price_ma12,price_pct_change,price_volatility
0,2010-01-01,24.370893,2010,1,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02-01,23.577170,2010,2,1,0,1,24.370893,NaN,NaN,NaN,NaN,NaN,NaN,-0.032568,NaN
2,2010-03-01,24.559611,2010,3,1,0,1,23.577170,NaN,NaN,NaN,24.169224,NaN,NaN,0.041669,NaN
3,2010-04-01,26.903787,2010,4,2,0,0,24.559611,24.370893,NaN,NaN,25.013523,NaN,NaN,0.095448,NaN
4,2010-05-01,24.707308,2010,5,2,1,0,26.903787,23.577170,NaN,NaN,25.390235,NaN,NaN,-0.081642,NaN
5,2010-06-01,27.207329,2010,6,2,1,0,24.707308,24.559611,NaN,NaN,26.272808,25.221016,NaN,0.101185,1.476716
6,2010-07-01,29.474016,2010,7,3,1,0,27.207329,26.903787,24.370893,NaN,27.129551,26.071537,NaN,0.083312,2.187621
7,2010-08-01,28.459293,2010,8,3,0,0,29.474016,24.707308,23.577170,NaN,28.380213,26.885224,NaN,-0.034428,1.971569
8,2010-09-01,24.413157,2010,9,3,0,0,28.459293,27.207329,24.559611,NaN,27.448822,26.860815,NaN,-0.142173,2.006713
9,2010-10-01,25.678200,2010,10,4,0,0,24.413157,29.474016,26.903787,NaN,26.183550,26.656551,NaN,0.051818,2.063050


## 4. Sauvegarde des données

In [10]:
# Sauvegarder le dataset final
output_path = PROCESSED_DATA_PATH / 'vanilla_prices.csv'
df_vanilla_features.to_csv(output_path, index=False)
print(f"✅ Dataset sauvegardé: {output_path}")

# Sauvegarder aussi une version sans NaN (pour les modèles)
df_clean = df_vanilla_features.dropna()
clean_path = PROCESSED_DATA_PATH / 'vanilla_prices_clean.csv'
df_clean.to_csv(clean_path, index=False)
print(f"✅ Dataset nettoyé sauvegardé: {clean_path}")
print(f"   {len(df_clean)} observations (après suppression NaN)")

✅ Dataset sauvegardé: ../data/processed/vanilla_prices.csv
✅ Dataset nettoyé sauvegardé: ../data/processed/vanilla_prices_clean.csv
   168 observations (après suppression NaN)


## 📋 Résumé

### Données collectées:
- **Période**: 2010-2024 (15 ans)
- **Fréquence**: Mensuelle
- **Observations**: 180 points

### Features créées:
- `price_usd_kg`: Prix cible (USD/kg)
- `year`, `month`, `quarter`: Temporelles
- `harvest_season`, `cyclone_season`: Indicateurs saisonniers
- `price_lag*`: Features de lag
- `price_ma*`: Moyennes mobiles
- `price_pct_change`, `price_volatility`: Indicateurs de tendance

### Prochaine étape:
→ Notebook `02_eda.ipynb` pour l'analyse exploratoire